In [9]:
from google.cloud import bigquery as bq
from google.cloud import bigquery_storage as bq_storage
import plotly.express as px
import pandas as pd

# create clients
client = bq.Client.from_service_account_json("key.json")
storage_client = bq_storage.BigQueryReadClient.from_service_account_json("key.json")

bq_dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
bq_dataset = client.get_dataset(bq_dataset_ref)


In [14]:
def strCont(string, x):
    return string.find(x) != -1

def setTech(x):
    if(strCont(x, 'react')):
        return 'react'
    elif(strCont(x, 'vue')):
        return 'vue'
    elif(strCont(x, 'angular')):
        return 'angular'
    elif(strCont(x, 'express')):
        return 'express'
    elif(strCont(x, 'django')):
        return 'django'
    elif(strCont(x, 'symfony')):
        return 'symfony'
    elif(strCont(x, 'ruby-on-rails')):
        return 'ruby-on-rails'
    elif(strCont(x, 'node')):
        return 'node.js'
    elif(strCont(x, 'asp')):
        return 'asp'
    elif(strCont(x, 'servlet') or strCont(x, 'jsp')):
        return 'java-servlet'
    elif(strCont(x, 'php')):
        return 'php'
    elif(strCont(x, 'html')):
        return 'html'
    elif(strCont(x, 'javascript') or strCont(x, 'js')):
        return 'javascript'
    elif(strCont(x, 'spring')):
        return 'spring'
    else:
        return

def creationDateToDay(date):
    return pd.to_datetime(date).to_period('M')

In [15]:
# Percent of questions on stack overflow relative to new and old technologies
# 1) from db take N questions, randomly, selecting tags and date.
# 2) create a new column named technlogy, based on tags 
# 3) substitute timestamp with month

In [20]:
#GET TAGS DATASET AND MODIFY

sample_size = 1000000 #--> 1 MLN

query = '''
    SELECT RAND() as r, tags, creation_date FROM `bigquery-public-data.stackoverflow.posts_questions`
    ORDER BY r
    LIMIT ''' + str(sample_size)


all_question = (
    client.query(query)
    .result()
    .to_dataframe(bqstorage_client=storage_client)
)

all_question = all_question.drop(columns=['r']).rename(columns={'tags':'technology'})
all_question['technology']=all_question['technology'].map(setTech)
all_question['month']=all_question['creation_date'].map(creationDateToDay)
all_question = all_question.drop(columns=['creation_date'])
all_question

,technology,month
0,none,2013-12
1,none,2018-10
2,none,2014-05
3,asp,2016-10
4,none,2011-04
...,...,...
999995,none,2015-11
999996,ruby-on-rails,2014-06
999997,none,2013-08
999998,none,2016-02


In [21]:
#SAVE TO CSV
all_question.to_csv('tags_dataframe.csv')